In [1]:
from tw_srch.data_extraction import data_extract
from keras.models import load_model
import pickle
import pandas as pd
import os
from keras.preprocessing.sequence import pad_sequences
import datetime
import mysql.connector

In [2]:
mydb = mysql.connector.connect(
    host='127.0.0.1',
    user="chordio",
    password="chordio",
    database="social_data"
)

In [3]:
def add_data(id,data,author,loc,cat,date):
    mycursor = mydb.cursor()
    edate = datetime.datetime.strptime(date, "%Y-%m-%d")
    parameters = [id,data,author,loc,cat,edate]
    insert_ur = "INSERT INTO `data`(`tweet_id`,`data`,`author_id`,`location`,`category`,`date`) values((%s),(%s),(%s),(%s),(%s),(%s))"
    mycursor.execute(insert_ur,parameters)
    mydb.commit()


In [4]:
model = load_model('model-v2.h5')
with open('tokenizer.pickle','rb') as token:
    tokenizer = pickle.load(token)

In [5]:
files = os.listdir('keywords')
dxt = data_extract()

In [6]:
category = {}
for file in files:
    key_file = open(f'keywords/{file}')
    keywords = key_file.read().split('\n')
    category[str(file[9:-4])] = keywords

In [7]:
def tokenize_pad_sequences(text):
    max_len=50
    val = [0,1]
    xt = tokenizer.texts_to_sequences(text)
    xt = pad_sequences(xt, padding='post', maxlen=max_len)
    yt = model.predict(xt).argmax(axis=1)
    return val[yt[0]]

In [8]:
while True:
    for cat in category:
        for words in category[cat]:
            results = dxt.get_data(words)
            for result in results:
                relevence = tokenize_pad_sequences([str(result[1])])
                if relevence == 0:
                    continue
                add_data(id=result[0],data=result[1],author=result[2],loc=result[3],date=result[4],cat=cat)